<a href="https://colab.research.google.com/github/DarthCoder501/GAAP/blob/main/Dataset_Creation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd


# Read the original CSV file
df = pd.read_csv('/content/image_ehr_crosswalk_20250418.csv')


# 1. Drop rows where note_id == -1
df = df[df['note_id'] != -1]


# Group by person_id and check for multiple unique image_ids
repeating_persons = df.groupby('person_id').filter(lambda x: x['image_id'].nunique() > 1)

In [ ]:
df.info()

In [ ]:
print(f"Found {len(repeating_persons)} records with repeating person_ids and different image_ids")

In [ ]:
# Save to a new CSV file
repeating_persons.to_csv('filtered_repeating_persons.csv', index=False)

In [ ]:
import pandas as pd

# Read both CSV files
df1 = pd.read_csv('/content/Final_Impressions.csv')  # contains 'impression_id'
df2 = pd.read_csv('/content/filtered_repeating_persons.csv')  # contains 'image_id' and 'note_id'

# First filter df2 to exclude rows with note_id == -1
df2_filtered = df2[df2['note_id'] != -1]


In [ ]:
df2_filtered = df2_filtered.drop("impression_id", axis=1)

In [ ]:
# Then rename image_id to impression_id for merging
df2_filtered = df2_filtered.rename(columns={'image_id': 'impression_id'})

# Perform an inner merge - this automatically keeps only impression_ids that exist in both
merged_df = pd.merge(
    df1,
    df2_filtered,
    on='impression_id',
    how='inner'
)


In [ ]:
merged_df

In [ ]:
merged_df = merged_df.drop("note_id", axis=1)

In [ ]:
merged_df

In [ ]:
# Save the result
merged_df.to_csv('filtered_merged_output.csv', index=False)

In [ ]:
# Load both CSVs
DF1 = pd.read_csv('/content/filtered_merged_output.csv')  # Keep rows from here
df2 = pd.read_csv('/content/Final_Impressions_labels.csv')  # Has the balanced impression_ids

In [ ]:
# Filter df1 to only keep rows where impression_id is in df2
filtered_df1 = df1[df1['impression_id'].isin(df2['impression_id'])]

In [ ]:
filtered_df1

In [ ]:
merged_df = pd.merge(
    DF1,
    df2,
    on='impression_id',
    how='inner'
)

In [ ]:
merged_df

In [ ]:
merged_df = merged_df.drop("1_month_mortality", axis=1)

In [ ]:
merged_df = merged_df.drop("6_month_mortality", axis=1)

In [ ]:
merged_df = merged_df.drop("12_month_mortality", axis=1)

In [ ]:
merged_df = merged_df.drop("12_month_PH", axis=1)

In [ ]:
merged_df = merged_df.drop("pe_acute", axis=1)

In [ ]:
merged_df = merged_df.drop("pe_subsegmentalonly", axis=1)

In [ ]:
merged_df

In [ ]:
# Value to drop
value_to_drop = 'Censored'

In [ ]:
# Create the boolean mask
mask = (merged_df['1_month_readmission'] != value_to_drop) & \
       (merged_df['6_month_readmission'] != value_to_drop) & \
       (merged_df['12_month_readmission'] != value_to_drop)

# Apply the mask to keep rows that don't contain 'Censored'
df_filtered = merged_df[mask]

In [ ]:
df_filtered

In [ ]:
# Function to identify boolean-like columns (True/False or 0/1)
def imbalance(series):
    return set(series.dropna().unique()).issubset({True, False, 0, 1, 'TRUE', 'FALSE', 'true', 'false'})

# Normalize values: convert TRUE/FALSE strings to bool
preprocess_df = df_filtered.replace({'TRUE': True, 'FALSE': False, 'true': True, 'false': False})

# Identify boolean-like columns (excluding ID)
bool_cols = [col for col in preprocess_df.columns if col != 'impression_id' and imbalance(df_filtered[col])]

# Count True vs False or 1 vs 0 in each boolean column
for col in bool_cols:
    counts = preprocess_df[col].value_counts(dropna=False)
    print(f"\nColumn: {col}")
    print(counts)
    print("Class Balance (%):")
    print(preprocess_df[col].value_counts(normalize=True, dropna=False) * 100)


Column: 1_month_readmission
1_month_readmission
False    5268
True      290
Name: count, dtype: int64
Class Balance (%):
1_month_readmission
False    94.782296
True      5.217704
Name: proportion, dtype: float64

Column: 6_month_readmission
6_month_readmission
False    4762
True      796
Name: count, dtype: int64
Class Balance (%):
6_month_readmission
False    85.678302
True     14.321698
Name: proportion, dtype: float64

Column: 12_month_readmission
12_month_readmission
False    4487
True     1071
Name: count, dtype: int64
Class Balance (%):
12_month_readmission
False    80.730479
True     19.269521
Name: proportion, dtype: float64

Column: pe_positive
pe_positive
0    4240
1    1318
Name: count, dtype: int64
Class Balance (%):
pe_positive
0    76.286434
1    23.713566
Name: proportion, dtype: float64


/tmp/ipython-input-30-3647954463.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  preprocess_df = df_filtered.replace({'TRUE': True, 'FALSE': False, 'true': True, 'false': False})


In [ ]:
df_filtered.to_csv("Progression Dataset.csv", index=False)